In [1]:
from textblob import TextBlob
import re
from nltk import word_tokenize 
from nltk import pos_tag
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from PIL import Image
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
df_azure_speech = pd.DataFrame(columns=['Sentence', 'Pre-processed sentence', 'textblob-subjectivity', 'vader-pos', 'vader-neg', 'vader-neu', 'vader-compound', 'vader-polarity', 'sentiment-category'])
pos_words_str_vader = ""
neg_words_str_vader = ""

In [3]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he shall have he will have",
"he's": "he has he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has how is",
"i'd": "I had I would",
"i'd've": "I would have",
"i'll": "I shall I will",
"i'll've": "I shall have I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had it would",
"it'd've": "it would have",
"it'll": "it shall it will",
"it'll've": "it shall have it will have",
"it's": "it has it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had she would",
"she'd've": "she would have",
"she'll": "she shall she will",
"she'll've": "she shall have she will have",
"she's": "she has she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as so is",
"that'd": "that would that had",
"that'd've": "that would have",
"that's": "that has that is",
"there'd": "there had there would",
"there'd've": "there would have",
"there's": "there has there is",
"they'd": "they had they would",
"they'd've": "they would have",
"they'll": "they shall they will",
"they'll've": "they shall have they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has what is",
"what've": "what have",
"when's": "when has when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has where is",
"where've": "where have",
"who'll": "who shall who will",
"who'll've": "who shall have who will have",
"who's": "who has who is",
"who've": "who have",
"why's": "why has why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had you would",
"you'd've": "you would have",
"you'll": "you shall you will",
"you'll've": "you shall have you will have",
"you're": "you are",
"you've": "you have"
}

In [4]:
sentiment_category = {
    'Satisfaction' : ['like', 'love', 'enjoy', 'appreciate'], # like - nejednoznacne 
    'Dissatisfaction' : ['hate', 'dislike'],
    'Confusion' : ['confuse', 'stuck', 'puzzle'],
    'Expectation' : ['expect', 'hope', 'anticipate', 'assume', 'suppose'], # think but... hope to find ... look for
    'Suggestion' : ['prefer', 'preferable', 'favor', 'suggest'] # 'like better', 'go for'
}

In [5]:
filename = "results-eval/18-sk-en-results.csv"
with open("speech/3_sk_en.txt", 'r') as transcript_sentences:
    text = transcript_sentences.read()

sentences = nltk.tokenize.sent_tokenize(text)

print(sentences)

for sentence in sentences:
    print(sentence)

["So I guess I'm supposed to get acquainted with this site in a minute, 2.", "I'm looking at the headline.", "Um, yes, so I see some statistics, then I'm going downstairs, it's something about vaccinations.", 'Quick lines, good map.', 'Hmm, 1 impression is.', 'Hi, it looks like.', 'This page looks trustworthy.', "I'm clear.", 'Highest number.', 'Metals tk isis good.', 'He was looking at the map.', "I'm a little confused about this because I don't know if I'm good, if I'm supposed to look for it in those advanced statistics.", 'I mean, in those expanded statistics?', 'Or should I look for it somewhere else entirely?', "I'm not sure now I'm even looking for it well, but that's how I look at the numbers.", 'Uh, I see here some antigen test results for some media in seven days.', 'Number of vaccinated.', 'Their relatives and then.', 'Map.', 'Um, which might have an answer to that three tasks.', 'Uh, the region will probably be.', 'Maybe these 99, so in which region?', 'Probably in bratisla

In [6]:
def clean_sentences(text):
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    text = re.sub(r'[^\w\s]', '', text)
    return text

lines_clean = []
for sentence in sentences:
    lines_clean.append(clean_sentences(sentence))
    
print(lines_clean)

['So I guess I am supposed to get acquainted with this site in a minute 2', 'I am looking at the headline', 'Um yes so I see some statistics then I am going downstairs it has it is something about vaccinations', 'Quick lines good map', 'Hmm 1 impression is', 'Hi it looks like', 'This page looks trustworthy', 'I am clear', 'Highest number', 'Metals tk isis good', 'He was looking at the map', 'I am a little confused about this because I do not know if I am good if I am supposed to look for it in those advanced statistics', 'I mean in those expanded statistics', 'Or should I look for it somewhere else entirely', 'I am not sure now I am even looking for it well but that has that is how I look at the numbers', 'Uh I see here some antigen test results for some media in seven days', 'Number of vaccinated', 'Their relatives and then', 'Map', 'Um which might have an answer to that three tasks', 'Uh the region will probably be', 'Maybe these 99 so in which region', 'Probably in bratislava', 'No 

In [7]:
pos_dict = {'J': wordnet.ADJ, 'V': wordnet.VERB, 'N': wordnet.NOUN, 'R': wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(str(text)))
    newlist = []
    for word, tag in tags:
        newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

In [8]:
lines_token_pos = []
for sentence in lines_clean:
    lines_token_pos.append(token_stop_pos(sentence))

print(lines_token_pos)

[[('So', 'r'), ('I', None), ('guess', 'v'), ('I', None), ('am', 'v'), ('supposed', 'v'), ('to', None), ('get', 'v'), ('acquainted', 'v'), ('with', None), ('this', None), ('site', 'n'), ('in', None), ('a', None), ('minute', 'n'), ('2', None)], [('I', None), ('am', 'v'), ('looking', 'v'), ('at', None), ('the', None), ('headline', 'n')], [('Um', 'n'), ('yes', None), ('so', 'r'), ('I', None), ('see', 'v'), ('some', None), ('statistics', 'n'), ('then', 'r'), ('I', None), ('am', 'v'), ('going', 'v'), ('downstairs', 'n'), ('it', None), ('has', 'v'), ('it', None), ('is', 'v'), ('something', 'n'), ('about', None), ('vaccinations', 'n')], [('Quick', 'a'), ('lines', 'n'), ('good', 'a'), ('map', 'n')], [('Hmm', None), ('1', None), ('impression', 'n'), ('is', 'v')], [('Hi', 'n'), ('it', None), ('looks', 'v'), ('like', None)], [('This', None), ('page', 'n'), ('looks', 'v'), ('trustworthy', 'a')], [('I', None), ('am', 'v'), ('clear', 'a')], [('Highest', 'n'), ('number', 'n')], [('Metals', 'n'), ('tk'

In [9]:
wordnet_lemmatizer = WordNetLemmatizer()     

In [10]:
def lemmatize_(text):
    lemmatized_sentence = []
    for word, tag in text:
        if tag is None:
            if word.lower() == "like":
                lemmatized_sentence.append(word + '-(NON-STANDARD)')
            else:
                lemmatized_sentence.append(word)
        else:       
            lemmatized_sentence.append(wordnet_lemmatizer.lemmatize(word, tag))
    #lemmatized_sentence = " ".join(lemmatized_sentence)
    
    return lemmatized_sentence

In [11]:
lines_lemma = []
for phrase in lines_token_pos:
    lines_lemma.append(lemmatize_(phrase))

print(lines_lemma)

[['So', 'I', 'guess', 'I', 'be', 'suppose', 'to', 'get', 'acquaint', 'with', 'this', 'site', 'in', 'a', 'minute', '2'], ['I', 'be', 'look', 'at', 'the', 'headline'], ['Um', 'yes', 'so', 'I', 'see', 'some', 'statistic', 'then', 'I', 'be', 'go', 'downstairs', 'it', 'have', 'it', 'be', 'something', 'about', 'vaccination'], ['Quick', 'line', 'good', 'map'], ['Hmm', '1', 'impression', 'be'], ['Hi', 'it', 'look', 'like-(NON-STANDARD)'], ['This', 'page', 'look', 'trustworthy'], ['I', 'be', 'clear'], ['Highest', 'number'], ['Metals', 'tk', 'isi', 'good'], ['He', 'be', 'look', 'at', 'the', 'map'], ['I', 'be', 'a', 'little', 'confused', 'about', 'this', 'because', 'I', 'do', 'not', 'know', 'if', 'I', 'be', 'good', 'if', 'I', 'be', 'suppose', 'to', 'look', 'for', 'it', 'in', 'those', 'advanced', 'statistic'], ['I', 'mean', 'in', 'those', 'expand', 'statistic'], ['Or', 'should', 'I', 'look', 'for', 'it', 'somewhere', 'else', 'entirely'], ['I', 'be', 'not', 'sure', 'now', 'I', 'be', 'even', 'look',

In [12]:
stopwords_list = stopwords.words('english')

In [13]:
unwanted = {'no', 'not', 'nor'}
custom_stopwords = [e for e in stopwords_list if e not in unwanted]

In [14]:
def remove_stopwords(text):
    clean_text = []
    for word in text:
        if word.lower() not in set(custom_stopwords):
            clean_text.append(word.lower())
    return clean_text

In [15]:
preprocessed_text = []
for phrase in lines_lemma:
    preprocessed_text.append(remove_stopwords(phrase))

print(preprocessed_text)

[['guess', 'suppose', 'get', 'acquaint', 'site', 'minute', '2'], ['look', 'headline'], ['um', 'yes', 'see', 'statistic', 'go', 'downstairs', 'something', 'vaccination'], ['quick', 'line', 'good', 'map'], ['hmm', '1', 'impression'], ['hi', 'look', 'like-(non-standard)'], ['page', 'look', 'trustworthy'], ['clear'], ['highest', 'number'], ['metals', 'tk', 'isi', 'good'], ['look', 'map'], ['little', 'confused', 'not', 'know', 'good', 'suppose', 'look', 'advanced', 'statistic'], ['mean', 'expand', 'statistic'], ['look', 'somewhere', 'else', 'entirely'], ['not', 'sure', 'even', 'look', 'well', 'look', 'number'], ['uh', 'see', 'antigen', 'test', 'result', 'medium', 'seven', 'day'], ['number', 'vaccinated'], ['relative'], ['map'], ['um', 'might', 'answer', 'three', 'task'], ['uh', 'region', 'probably'], ['maybe', '99', 'region'], ['probably', 'bratislava'], ['no', 'not', 'catch', 'well', 'beremed', 'back', 'answer', 'preĺˇov', 'region', 'look', 'totalitarianism', 'know', '259404'], ['lead', 'p

In [16]:
analyzer = SentimentIntensityAnalyzer()

In [17]:
new_words = {
    'see': 2.0,
    'find': 2.0,
    'spot': 2.0,
    'grasp': 2.0,
    'perceive': 2.0,
    'identify': 2.0,
    'notice': 2.0,
    'distinguish': 2.0,
    'sight': 2.0,
    'glimpse': 2.0,
    'recognize': 2.0,
    'follow': 2.0,
    'realize': 2.0,
    'discover': 2.0,
    'little': -2.0,
    'small': -2.0,
    'tiny': -2.0,
    'large': 2.0,
    'know': 2.0, 
    'understand': 2.0,
    'kind': 1.5,
    'pretty': 1.5,
    'inconsistent': -2.0
}

analyzer.lexicon.update(new_words)

In [18]:
def search(values, searchFor):
    cat_list = []
    for word in searchFor:
        for k in values:
            for v in values[k]:
                if word == v:
                    cat_list.append(k)
                    #return k
    if cat_list:
        return cat_list
    return None

In [19]:
df_azure_speech = pd.DataFrame(columns=['Sentence', 'Pre-processed sentence', 'textblob-subjectivity', 'vader-pos', 'vader-neg', 'vader-neu', 'vader-compound', 'vader-polarity', 'sentiment-category'])
pos_words_str_vader = ""
neg_words_str_vader = ""

In [20]:
for sentence, line in zip(sentences, preprocessed_text):
    line_string = " ".join(line)
    txtblob_object = TextBlob(line_string)
    subjectivity = txtblob_object.sentiment.subjectivity
    sentiment_vader = analyzer.polarity_scores(line_string)

    sent_cat = search(sentiment_category, line)
    
    if sentiment_vader['compound'] < 0:
        polarity = 'Negative'
        neg_words_str_vader += line_string
        
    elif sentiment_vader['compound'] > 0:
        polarity = 'Positive'
        pos_words_str_vader += line_string
    else:
        polarity = 'Neutral'
    df_azure_speech = df_azure_speech.append({'Sentence': sentence,'Pre-processed sentence': line_string, 'textblob-subjectivity': subjectivity, 'vader-pos': sentiment_vader['pos'], 'vader-neg': sentiment_vader['neg'], 'vader-neu':sentiment_vader['neu'], 'vader-compound':sentiment_vader['compound'], 'vader-polarity': polarity, 'sentiment-category': sent_cat}, ignore_index=True)
  

In [21]:
df_azure_speech.to_csv(filename)

FileNotFoundError: [Errno 2] No such file or directory: 'results-eval/18-sk-en-results.csv'

In [ ]:
wordcloud = WordCloud(font_path = r'C:\Windows\Fonts\Verdana.ttf',
                            stopwords = STOPWORDS,
                            background_color = 'white',
                            width = 1200,
                            height = 1000,
                            ).generate(neg_words_str_vader)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
wordcloud = WordCloud(font_path = r'C:\Windows\Fonts\Verdana.ttf',
                            stopwords = STOPWORDS,
                            background_color = 'white',
                            width = 1200,
                            height = 1000,
                            ).generate(pos_words_str_vader)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
# def sentiment_analysis(textfile):
#     with open(textfile) as transcript:
#         lines = transcript.readlines()
#     lines_clean = clean(lines[0])
#     lines_token_stop_pos = token_stop_pos(lines_clean)
#     lines_lemma = lemmatize(lines_token_stop_pos).lower()
#     ngram_object = TextBlob(lines_lemma)
#     ngrams = ngram_object.ngrams(n=3) 
#     for ngram in ngrams:
#         ngram_str = TextBlob(' '.join(ngram))
#         sentiment = ngram_str.sentiment.polarity
#         subjectivity = ngram_str.sentiment.subjectivity
#         # sentiment_vader = analyzer.polarity_scores(ngram_str)
#         if sentiment >= 0.2:
#             print('Ngram: ' + ' '.join(ngram) + ' | sentiment: ' + str(sentiment) + ' | subjectivity: ' + str(subjectivity))
#         if sentiment < 0:
#             print('Ngram: ' + ' '.join(ngram) + ' | sentiment: ' + str(sentiment) + ' | subjectivity: ' + str(subjectivity))
    
# #     file_content=open (textfile).read()

# #     wordcloud = WordCloud(font_path = r'C:\Windows\Fonts\Verdana.ttf',
# #                             stopwords = STOPWORDS,
# #                             background_color = 'white',
# #                             width = 1200,
# #                             height = 1000,
# #                             color_func = random_color_func
# #                             ).generate(lines_lemma)
# #     plt.imshow(wordcloud)
# #     plt.axis('off')
# #     plt.show()

In [ ]:
# sentiment_analysis("testing_text.txt")

In [ ]:
df_azure_speech

In [ ]:
df_azure_speech.hist()

In [ ]:
sns.boxplot(data=df_azure_speech, x="vader-polarity", y="vader-compound")

In [ ]:
df_azure_speech['vader-polarity'].value_counts().plot(kind='pie')

In [ ]:
df_azure_speech.sort_values('vader-compound').head(20)

In [ ]:
df_azure_speech.sort_values('vader-compound', ascending=False).head(20)